# **ETL**

In [1]:
# Main libraries

import requests 
import os 
import pandas as pd  
import numpy as np
from datetime import datetime
from meteostat import Point, Daily, Hourly
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
pd.set_option('display.max_rows', 5)

In [22]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
    


## **Weather Readings: Mexico city**

**Method**: Meteostat Python Library 

In [7]:

MxC_latitude = 19.4333   
MxC_longitude = -99.1333

MxC = Point(MxC_latitude, MxC_longitude)

**Daily data**

In [9]:
# Set time period
start = datetime(1994, 1, 1)
end = datetime(2024, 11, 3)

# Get daily data
MxC_daily = Daily(MxC, start, end)
MxC_daily = MxC_daily.fetch()
MxC_daily

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2007-06-29,17.6,13.5,23.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-06-30,18.1,11.7,24.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-11-02,17.6,13.0,23.0,0.0,NaN,12.0,11.0,NaN,1024.5,NaN
2024-11-03,18.1,12.0,25.0,0.0,NaN,274.0,6.1,NaN,1015.8,NaN


**Hourly data**

In [ ]:
# Data exploration
MxC_hourly = Hourly(MxC, start, end)
MxC_hourly = MxC_hourly.fetch()
MxC_hourly

In [12]:
# Adding columns according to the ER diagram 
MxC_hourly['date']=MxC_hourly.index.date 
MxC_hourly['time']=MxC_hourly.index.time 
MxC_hourly['city']= 'Mexico city'
MxC_hourly

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,date,time,city
time,,,,,,,,,,,,,,
1994-01-01 21:00:00,18.5,4.3,39.0,NaN,NaN,30.0,3.6,NaN,1012.2,NaN,NaN,1994-01-01,21:00:00,Mexico city
1994-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01,22:00:00,Mexico city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 23:00:00,20.0,9.0,49.0,0.0,NaN,110.0,19.0,NaN,1023.0,NaN,8.0,2024-11-02,23:00:00,Mexico city
2024-11-03 00:00:00,18.0,11.1,64.0,0.0,NaN,70.0,13.0,NaN,1024.0,NaN,8.0,2024-11-03,00:00:00,Mexico city


In [13]:
# Renaming columns according to the ER diagram 
MxC_hourly = MxC_hourly.rename(columns={'temp':'temperature','coco':'code', 'dwpt':'dew_point', 'pres':'pressure', 'wspd':'wind_speed','rhum':'relative_humidity'})
MxC_hourly

,temperature,dew_point,relative_humidity,prcp,snow,wdir,wind_speed,wpgt,pressure,tsun,code,date,time,city
time,,,,,,,,,,,,,,
1994-01-01 21:00:00,18.5,4.3,39.0,NaN,NaN,30.0,3.6,NaN,1012.2,NaN,NaN,1994-01-01,21:00:00,Mexico city
1994-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01,22:00:00,Mexico city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 23:00:00,20.0,9.0,49.0,0.0,NaN,110.0,19.0,NaN,1023.0,NaN,8.0,2024-11-02,23:00:00,Mexico city
2024-11-03 00:00:00,18.0,11.1,64.0,0.0,NaN,70.0,13.0,NaN,1024.0,NaN,8.0,2024-11-03,00:00:00,Mexico city


In [15]:
# Filtering needed columns
MxC_weather_readings=MxC_hourly[['date','time','temperature','dew_point','pressure','wind_speed', 'relative_humidity']]
MxC_weather_readings

,date,time,temperature,dew_point,pressure,wind_speed,relative_humidity
time,,,,,,,
1994-01-01 21:00:00,1994-01-01,21:00:00,18.5,4.3,1012.2,3.6,39.0
1994-01-01 22:00:00,1994-01-01,22:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-11-02 23:00:00,2024-11-02,23:00:00,20.0,9.0,1023.0,19.0,49.0
2024-11-03 00:00:00,2024-11-03,00:00:00,18.0,11.1,1024.0,13.0,64.0


In [ ]:
# Database Connection: We use sql server with a trusted connection and weather as database name
engine = create_engine('mssql+pyodbc://SERVER_NAME/weather?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')


In [17]:
# Database load
MxC_weather_readings.to_sql('weather_readings', engine, if_exists='append', index=False)

20

In [19]:
# Filtering needed columns
MxC_city_codes = MxC_hourly[['date','time','code', 'city','temperature','dew_point','pressure','wind_speed', 'relative_humidity']]
MxC_city_codes

,date,time,code,city,temperature,dew_point,pressure,wind_speed,relative_humidity
time,,,,,,,,,
1994-01-01 21:00:00,1994-01-01,21:00:00,NaN,Mexico city,18.5,4.3,1012.2,3.6,39.0
1994-01-01 22:00:00,1994-01-01,22:00:00,NaN,Mexico city,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-11-02 23:00:00,2024-11-02,23:00:00,8.0,Mexico city,20.0,9.0,1023.0,19.0,49.0
2024-11-03 00:00:00,2024-11-03,00:00:00,8.0,Mexico city,18.0,11.1,1024.0,13.0,64.0


In [21]:
# Database load
MxC_city_codes.to_sql('w_readings_city_codes', engine, if_exists='append', index=False)


36